### This Notebook demonstrates how to use 23ai Vector Search and Select AI
<br><br> 
<dl>
  &nbsp;   &nbsp;   &nbsp;   &nbsp;   &nbsp; &nbsp;   &nbsp;   &nbsp;   &nbsp;   &nbsp; &nbsp;   &nbsp;   &nbsp;   &nbsp;   &nbsp; 
   <img src="https://blogs.oracle.com/content/published/api/v1.1/assets/CONT07CB11DCD0A048D88730F792EDB7A38C/Medium?cb=_cache_47af&channelToken=af5d837e3d34400dbe9ae6cec73ee9b7&format=jpg" alt="OML Notebooks" width="900"/>
</dl>

https://blogs.oracle.com/database/post/oracle-announces-general-availability-of-ai-vector-search-in-oracle-database-23ai
<br>
<br>
https://blogs.oracle.com/machinelearning/post/accelerate-innovation-with-select-ai-and-oci-generative-ai

Autonomous Database makes it simple to vectorize data with Oracle 23ai Vector Search and Oracle Machine Learning (OML). 
<br> <br>
The following paragraphs will show you how to:
<br>
1. Create Credentials
2. Import a model to vectorize data
3. Create a table with text data (from external table)
4. Vectorize text columns
5. Show the SQL generated for a query

As Admin user start doing the following:
<br>
1. grant execute on dbms_cloud to "USER"
2. grant execute on dbms_cloud_ai to "USER" 
3. grant create mining model to "USER" -- or -- grant select any mining model to "USER"

#### Create credential

Below is the code used to create credential to read from object_storage and to sign LLM API requests in OCI. This code has already been run for my user. 


    begin
        dbms_cloud.create_credential (
            credential_name => 'OCI_CRED',
            user_ocid => '....',
            tenancy_ocid => '....',
            private_key => '....',
            fingerprint => '....'
        );
    end;



<br>Below is the code used to create a credential for OpenAI LLM. 


    begin
        dbms_cloud.create_credential (
            credential_name  => 'OPENAI_CRED',
            username         => 'OPENAI',
            password         => 'your-llm-secret-goes-here'
              );
    end;


##### NB! *  next to the title means that you must have created credential and an AI profile

<dl>
  &nbsp;   &nbsp;   &nbsp;   &nbsp;   &nbsp; &nbsp;   &nbsp;   &nbsp;   &nbsp;   &nbsp; &nbsp;   &nbsp;   &nbsp;   &nbsp;   &nbsp; 
   <img src="https://docs.oracle.com/en/database/oracle/oracle-database/23/vecse/img/ai_vector_workflow_ga_v2.png" alt="Oracle AI Vector Search Use Case Flowchart" width="1200"/>
</dl>

#### Import ONNX (Open Neural Network eXchange) embedding model from Object Storage

Below is the code used to read onnx objects stored in object_storage.


    Select * FROM 
        DBMS_CLOUD.LIST_OBJECTS(
            'OCI_CRED', 'https://objectstorage.eu-frankfurt-1.oraclecloud.com/n/.....')
    where object_name like '%onnx';

<br><br>
Below is the code used to store onnx model in OML in the Oracle database.


    DECLARE
        model_source BLOB := NULL;
    begin
        model_source:= DBMS_CLOUD.GET_OBJECT(
            credential_name => 'OCI_CRED',
            object_uri => 'https://objectstorage.eu-frankfurt-1.oraclecloud.com/n/....'
            ); 
        
        DBMS_DATA_MINING.IMPORT_ONNX_MODEL(
            'huggingface',
            model_source, 
            JSON('{ "function" : "embedding",
            "embeddingOutput"  : "embedding",
            "input": {"input"  : ["DATA"]} }')
            );
    end;

I'am using 
####  all-MiniLM-L6-v2 
See blog: <br> 
 https://blogs.oracle.com/machinelearning/post/oml4py-leveraging-onnx-and-hugging-face-for-advanced-ai-vector-search

This is the ONNX-ported version of the sentence-transformers/all-MiniLM-L6-v2 for generating text embeddings. <br> 
It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for semantic search, where we look for sentences that have similar meanings. <br>

https://huggingface.co/onnx-models/all-MiniLM-L6-v2-onnx#onnx-modelsall-minilm-l6-v2-onnx

<br> 
Model details

    Embedding dimension: 384
    Max sequence length: 256
    File size: 0.09 GB

This is the ONNX-ported version of the sentence-transformers/all-MiniLM-L6-v2 for generating text embeddings. <br> 
It maps sentences & paragraphs to a 384 dimensional dense vector space and can be used for semantic search, where we look for sentences that have similar meanings. <br>
<br> 
<p>You can also test this model </p>
<h4> all-MiniLM-L12-v2 </h4>
See blog: <br> 
 <a href="https://blogs.oracle.com/machinelearning/post/use-our-prebuilt-onnx-model-now-available-for-embedding-generation-in-oracle-database-23ai">https://blogs.oracle.com/machinelearning/post/use-our-prebuilt-onnx-model-now-available-for-embedding-generation-in-oracle-database-23ai</a>

#### Amazon Food Reviews dataset
The dataset consists of reviews of fine foods from amazon downloaded from <br>
https://huggingface.co/datasets/jhan21/amazon-food-reviews-dataset
<br>
1. Downlod the reviews file in csv format (the file consists of 568454 reviews)
2. Upload the csv file to the bucket in object_storage
3. Create an external table
<br>

#### External table
Below is the code used to create an external table from datafile in object_storage.
<br>

    begin
        DBMS_CLOUD.CREATE_EXTERNAL_TABLE(
            table_name =>'FOODREVIEWSFILE',
            credential_name =>'OCI_CRED',
            file_uri_list =>'https://objectstorage.eu-frankfurt-1.oraclecloud.com/n/......',
            format => json_object('type' VALUE 'csv', 'skipheaders' VALUE '1'),
            column_list =>'Id NUMBER,
                   ProductId varchar2(100),
                   UserId varchar2(100),
                   ProfileName varchar2(200),
                   HelpfulnessNumerator NUMBER,
                   HelpfulnessDenominator NUMBER,
                   Score NUMBER,
                   Time NUMBER,
                   Summary VARCHAR2(4000), 
                   Text VARCHAR2(32767)
        ');
    end;

<br>

#### View with 2% of the data from the external table
4. Create a view with 2% of the data and with 5 cloumns from the external table: ID, PRODUCTID, SCORE, SUMMARY and TEXT 
<br>

Below is the code used to create the view
<pre><code>create or replace view food_reviews_2 
    as select ID, PRODUCTID, SCORE, SUMMARY, TEXT from foodreviewsfile sample(2)
;
</code></pre>
<br>
Column details
<pre><code>
    Id: Unique identifier for each reviews
    ProductId: Unique identifier for the product
    Score: Rating between 1 and 5
    Summary: Brief summary of the review
    Text: Text of the review
</code></pre>

Creating the table with text column also stored as vector.


    create table foodreviews as
        (SELECT id, PRODUCTID, TEXT, Vector_embedding
            (HUGGINGFACE using  text as data) as vec 
            from FOOD_REVIEWS_2)
    ;

Profiles capture the properties of your LLM provider plus the tables and views you want to enable for natural language queries. 
You can create multiple profiles (e.g. for different providers) - each pointing to different models.

Autonomous Database makes it simple to query your data using natural language. 

The following paragraphs will show you how to:
1. Create an AI profile 
2. Set the profile for your session
3. Demo Vector Search
4. Have general AI chat conversations
5. Use chat to summarize a few of the reviews
6. Query data using natural language
7. Show the SQL generated for a query


#### Create AI Profile

Below is the code used to create credential to read from object_storage and to sign LLM API requests in OCI. This code has already been run for my user.


    begin
        -- ~~~~~~~~~~~~~~
        -- drop the AI profile just in case it already exists
        -- ~~~~~~~~~~~~~~
        dbms_cloud_ai.drop_profile(
            profile_name => 'GENAIP1_OCI',
            force => true
        );

        -- ~~~~~~~~~~~~~~
        -- Specify the LLM provider, your credential
        -- and the tables/views that will be used for queries
        -- You can specify specific tables. Or, omit the table from the object list to
        -- include all objects in the schema
        -- ~~~~~~~~~~~~~~   
        DBMS_CLOUD_AI.CREATE_PROFILE('GENAIP1_OCI', 
         '{"provider": "oci",  
         "credential_name": "OCI_CRED",
         "comments":"true",
        "object_list": [{"owner": "WSDEV04", "name": "foodreviews"}]
        }'); 
    end;

You can now ask questions using `SELECT AI`. **AI** is a special keyword in the select statement that tells Autonomous Database that the subsequent text will be either or an action or the natural language question.

Here are the actions:
1. chat    - general AI chat
2. runsql  - [default] ask a question and get a structured result
3. narrate - ask a question and get a conversational result
4. showsql - SQL used to produce the result

Enough explaining. Just give it a try!

Start by asking a general question that is not going to be answered by our data. But, the answer may lead to questions that we want to ask of our data:

You can use a PLSQL API to access the LLM. This is useful for tools that are unable to use the SQL interface. Here's a couple examples that use `DBMS_CLOUD_AI.GENERATE` that to translate natural lange to SQL.

This approach can be used by APEX app.

We described the different actions above: runsql, narrate, showsql, explainsql and chat. Whereas chat is just a general question, narrate and runsql are questions about our data. Take a look at the results from "FOODREVIEWS" in different actions:

COLUMN_NAME, DATA_TYPE:
 1. ID, NUMBER 
 2. PRODUCTID, VARCHAR2 
 3. TEXT, VARCHAR2 
 4. VEC, VECTOR